# Forest Fire Model

Make it possible to set the probability parameters p, f, q. Start with an empty field (all cells
are in state Ashes).
Write the number of cells for each state (Ashes, Tree, Fire) for each time step into a file
(three values per line, ASCII format, separated by blanks).

q - rate of induced growth\
f - rate of spontaneous fire\
p - rate of spontaneous growth

In [2]:
import numpy as np

In [3]:
q = 1 #1 - induced growth. 
f = 0.1
p = 0.5

In [4]:
rows, cols = (101, 82)
automaton = [[0]*cols]*rows
iterations = 20

### Auxilary functions

In [10]:
def process_singular_cell(cell):
    if cell == 0:
        return np.random.choice([0, 1], p = [1-p, p]) #creation of forest cell with a probability of p
    elif cell == 1:
        return np.random.choice([1, 2], p = [1-f, f]) #spreading fire with a probability of f
    elif cell == 2: 
        return 0 #cell that burned turned to ashes.

In [ ]:
def process_cell_neighbourhood(cell, neighbourhood_cell):
    if cell == 0:
        if neighbourhood_cell == 1:
            return np.random.choice([0, 1], p = [1-q, q])
        else:
            return 0
    elif cell == 1:
        if neighbourhood_cell == 2:
            return 2
        else:
            return 1


In [19]:
def neighbourhood_range(row_id, column_id, radius):
    """
    Returns the range of the neighbourhood of a given point in a 2D matrix.

    Args:
    - X: the number of rows in the matrix
    - Y: the number of columns in the matrix
    - row_id: the row index of the point
    - column_id: the column index of the point
    - radius: the radius of the neighbourhood (in number of cells away from the center)

    Returns:
    - A tuple (-1, 1, -1, 1) representing the range of the neighbourhood.
    """

    start_row = max(0, row_id - radius)
    end_row = min(rows - 1, row_id + radius)

    start_column = max(0, column_id - radius)
    end_column = min(cols - 1, column_id + radius)

    # Handle the special case where row_id is 0
    if row_id == 0:
        start_row = 0

    # Handle the special case where column_id is 0
    if column_id == 0:
        start_column = 0

    # Handle the special case where row_id is at the edge of the matrix
    if row_id == rows - 1:
        end_row = rows - 1

    # Handle the special case where column_id is at the edge of the matrix
    if column_id == cols - 1:
        end_column = cols - 1

    return (start_row - row_id, end_row - row_id, start_column - column_id, end_column - column_id)


In [30]:
(start_row, end_row, start_column, end_column) = neighbourhood_range(5, 5, 1)


for i in range(start_row, end_row+1):
            for j in range(start_column, end_column+1):
                if (j != 0 and i!=0) or (j == 0 and i ==0):
                    continue
                print(i,j)


-1 0
0 -1
0 1
1 0


In [8]:
def colored_string(numbers):
    color_map = {"0": "\033[0;33m", "1": "\033[32m", "2": "\033[31m"}
    colored_str = ""
    for n in numbers:
        if str(n) in color_map:
            colored_str += color_map[str(n)] + str(n) + "\033[0m"
        else:
            colored_str += str(n)
    return colored_str

In [9]:
print(colored_string("000111222"))

000111222


Let 0 be ashes (yellow), 1 - forest (green) and 2 - fire (red)

In [ ]:
running = True
iters = 0
while running:
    iters+=1
    for row_id, rows in enumerate(automaton):
        for column_id, cell in enumerate(rows):
            row_range_b, row_range_upper, column_range_b, column_range_upper = neighbourhood_range(row_id, column_id, 1)
            for i in range(row_range_b, row_range_upper+1):
                for j in range(column_range_b, column_range_upper):
                    if (j != 0 and i!=0) or (j == 0 and i ==0):
                        continue
                    cell = process_cell_neighbourhood(cell, automaton[i, j])
                    cell = process_singular_cell(cell)
                    automaton[row_id][column_id] = cell

    if iters == iterations:
        running = False